# Power spectra to inspect .dat files and data integrity

This script can be used to look at the beginning and end of .dat files to make sure that the files are not corrupted (e.g., unplugged cable, faulty amplifiers, etc.).

We create a pdf file with power spectrum of every channel, using the beginning and end of each .dat files. The results is displayed as a 2D image.
In the hippocampus, we expect a peak near 8 Hz (theta).

I also display 1 sec of data at the beginning and end of each channel.

In [2]:
%load_ext autoreload
%autoreload 2

import sys
from PyPDF2 import PdfFileReader, PdfFileWriter


from spikeA.Dat_file_reader import Dat_file_reader
from scipy import signal
# This sets up our session lists
%run setup_project.py
print("autpipy sessions: {}, spikeA sessions: {}".format(len(myProject.sessionList),len(sSesList)))

Project name: autopi_ca1
dataPath: /adata/projects/autopi_ca1
dlcModelPath: /adata/models
Reading /adata/projects/autopi_ca1/sessionList
We have 36 testing sessions in the list
See myProject and sSesList objects
autpipy sessions: 36, spikeA sessions: 36


In [3]:
def prepareSpikeASession(sSes):
    sSes.load_parameters_from_files() 
for sSes in sSesList:
    prepareSpikeASession(sSes)

In [4]:
sSesLocal = get_local_sessions(sSesList)
ip=socket.gethostname()
print("{} sessions are local to {}".format(len(sSesLocal),ip))

30 sessions are local to a230-pc73


There is one function to calculate the power spectra and get 1 sec of data to display. 
There are two functions to display the results.
There is one function to merge pdf files.

In [53]:
def calculate_power_spectra (sSes,nfft = 2**15,data_length_sec = 120):
    """
    Function to calculate power spectrum for the beginning and end of each .dat files
    
    The power spectra are returned in a np.array[files,channels,beg-end,frequencies]
    
    Arguments:
    sSes: spikeA session
    nfft: length of the data segment that goes into the fft
    data_length_sec: how much data to include from the beginning or from the end
    
    Returns:
    dat_files: names of the .dat files used 
    f: 1D np.array with frequencies
    res: 4D np.arrays with power spectra
    sig: 4D np.array with 1 second of signal from beginn or from the end
    """
    sampling_rate=int(sSes.sampling_rate)
    data_length = sampling_rate*data_length_sec
    dat_files = sSes.file_names["dat"]
    dfr = Dat_file_reader(dat_files,sSes.n_channels)
    
    ind = dfr.get_file_duration_seconds()>data_length_sec
    if np.sum(~ind) > 0:
        print(" {} is smaller than {} sec, length: {}".format(list(compress(dat_files, ~ind)),data_length_sec,dfr.get_file_duration_seconds()[~ind]))
        # recreate the dfr object without offending file
        dat_files=list(compress(dat_files, ind))
        dfr = Dat_file_reader(dat_files,sSes.n_channels)
    
    beg,end = dfr.get_first_last_samples_each_file()
    channels= np.arange(sSes.n_channels-1) # get one channel per probe from channel_map

    # array with [files,channels,beg-end,frequencies]
    res = np.empty((len(dat_files),channels.shape[0],2,int(nfft/2+1)))
    sig = np.empty((len(dat_files),channels.shape[0],2,int(sampling_rate)))

    for i,(b,e) in tqdm(enumerate(zip(beg,end))):
        # get the beginning ps
        dat = dfr.get_data_one_block(b,b+data_length,channels)
        for j in range(channels.shape[0]):
            f, Pxx_den = signal.periodogram(dat[j,:], sampling_rate, nfft=nfft, scaling="spectrum")
            res[i,j,0,:]=Pxx_den/np.max(Pxx_den)
            sig[i,j,0,:]=dat[j,0:sampling_rate]
        # get the end ps
        dat = dfr.get_data_one_block(e-data_length,e,channels)
        for j in range(channels.shape[0]):
            f, Pxx_den = signal.periodogram(dat[j,:], sampling_rate, nfft=nfft, scaling="spectrum")
            res[i,j,1,:]=Pxx_den/np.max(Pxx_den)
            sig[i,j,1,:]=dat[j,data_length-sampling_rate:data_length]
    return dat_files, f, res, sig

def power_spectra_plot(sSes,res,f,dat_files,min_freq=0.2,max_freq=100):
    
    col= 2
    row=len(dat_files)
    sizePerCol=6.0
    sizePerRow=3.0
    channels= np.arange(sSes.n_channels-1) # get one channel per probe from channel_map
    
    fig = plt.figure(figsize=(col*sizePerCol,row*sizePerRow),constrained_layout=False) # create the overall figure to put all the axes on
    gs = fig.add_gridspec(nrows=row, ncols=col,
                            wspace=0.2,hspace=0.5)
    freq_indices = np.logical_and(f>min_freq,f<max_freq)
    extent = [np.min(f[freq_indices]) , np.max(f[freq_indices]),0 , res.shape[1]]

    fn = sSes.path+"/channel_map.npy"
    channel_map = np.load(fn)
   
    for i in range(len(dat_files)):
        for j in range(2):
            ax = fig.add_subplot(gs[i,j])
            ax.imshow(res[i,channel_map,j,freq_indices],cmap="jet",aspect="auto",extent=extent)
            ax.set_xlabel("Frequency (Hz)")
            ax.set_ylabel("Channel")
            if j == 0:
                ax.set_title(sSes.trial_names[i]+".dat beginning")
            else:
                ax.set_title(sSes.trial_names[i]+".dat end")
    fn=sSes.path+"/inspect_power_spectra.pdf"
    print("Saving figure to "+fn)
    plt.savefig(fn)
    plt.close()
    #plt.show()
    return fn

def signal_plot(sSes,sig,dat_files):
    
    col= 2
    row=len(dat_files)
    sizePerCol=6.0
    sizePerRow=3.0
    channels= np.arange(sSes.n_channels-1) # get one channel per probe from channel_map
    
    fig = plt.figure(figsize=(col*sizePerCol,row*sizePerRow),constrained_layout=False) # create the overall figure to put all the axes on


    gs = fig.add_gridspec(nrows=row, ncols=col,
                            wspace=0.2,hspace=0.5)
    extent = [0 , sig.shape[-1],0/sSes.sampling_rate , len(channels)]
    
    fn = sSes.path+"/channel_map.npy"
    channel_map = np.load(fn).flatten()
    
    for i in range(len(dat_files)):
        for j in range(2):
            ax = fig.add_subplot(gs[i,j])
            ax.imshow(sig[i,channel_map,j,:],cmap="jet",aspect="auto",extent=extent)
            ax.set_xlabel("Time")
            ax.set_ylabel("Channel")
            if j == 0:
                ax.set_title(sSes.trial_names[i]+".dat beginning")
            else:
                ax.set_title(sSes.trial_names[i]+".dat end")
    fn=sSes.path+"/inspect_signal.pdf"
    print("Saving figure to "+fn)
    plt.savefig(fn)
    plt.close()
   # plt.show()
    return fn


def merge_pdf_files(input_files,output_file):
    """
    Merge some pdf files into 1
    
    Arguments
    input_files: list of file names to merge
    output_file: file names to save the merged pdf file
    """
    input_streams = []
    print(output_file)
    try:
        # First open all the files, then produce the output file, and
        # finally close the input files. This is necessary because
        # the data isn't read from the input files until the write
        # operation. Thanks to
        # https://stackoverflow.com/questions/6773631/problem-with-closing-python-pypdf-writing-getting-a-valueerror-i-o-operation/6773733#6773733
        output_stream = open(output_file,"bw")
        for input_file in input_files:
            input_streams.append(open(input_file, 'rb'))
        writer = PdfFileWriter()
        for reader in map(PdfFileReader, input_streams):
            for n in range(reader.getNumPages()):
                writer.addPage(reader.getPage(n))
        writer.write(output_stream)
    finally:
        for f in input_streams:
            f.close()
        output_stream.close()

def inspect_spectra_signal(sSes):
    dat_files,f, res,sig = calculate_power_spectra(sSes)
    fn1=power_spectra_plot(sSes,res,f,dat_files)
    fn2 = signal_plot(sSes,sig,dat_files)
    merge_pdf_files([fn1,fn2],sSes.path+"/inspect_summary.pdf")    

In [54]:
sSes = sSesLocal[1]
print(sSes.path)
inspect_spectra_signal(sSes)

/adata/projects/autopi_ca1/mn5824/mn5824-22112020-0107


7it [00:03,  2.21it/s]


Saving figure to /adata/projects/autopi_ca1/mn5824/mn5824-22112020-0107/inspect_power_spectra.pdf
Saving figure to /adata/projects/autopi_ca1/mn5824/mn5824-22112020-0107/inspect_signal.pdf
/adata/projects/autopi_ca1/mn5824/mn5824-22112020-0107/inspect_summary.pdf


In [55]:
for i,sSes in enumerate(sSesLocal[:]): # the last few sessions
    print(i,sSes.name)
    inspect_spectra_signal(sSes)

CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 2.86 µs
0 mn5824-20112020-0107


7it [00:03,  1.83it/s]


Saving figure to /adata/projects/autopi_ca1/mn5824/mn5824-20112020-0107/inspect_power_spectra.pdf
Saving figure to /adata/projects/autopi_ca1/mn5824/mn5824-20112020-0107/inspect_signal.pdf
/adata/projects/autopi_ca1/mn5824/mn5824-20112020-0107/inspect_summary.pdf
1 mn5824-22112020-0107


7it [00:03,  2.23it/s]


Saving figure to /adata/projects/autopi_ca1/mn5824/mn5824-22112020-0107/inspect_power_spectra.pdf
Saving figure to /adata/projects/autopi_ca1/mn5824/mn5824-22112020-0107/inspect_signal.pdf
/adata/projects/autopi_ca1/mn5824/mn5824-22112020-0107/inspect_summary.pdf
2 mn5824-24112020-0107


...

Saving figure to /adata/projects/autopi_ca1/TYY5622/TYY5622-07092021-0106/inspect_power_spectra.pdf
Saving figure to /adata/projects/autopi_ca1/TYY5622/TYY5622-07092021-0106/inspect_signal.pdf
/adata/projects/autopi_ca1/TYY5622/TYY5622-07092021-0106/inspect_summary.pdf
28 TYY5622-17092021-0106


6it [00:56,  9.44s/it]


Saving figure to /adata/projects/autopi_ca1/TYY5622/TYY5622-17092021-0106/inspect_power_spectra.pdf
Saving figure to /adata/projects/autopi_ca1/TYY5622/TYY5622-17092021-0106/inspect_signal.pdf
/adata/projects/autopi_ca1/TYY5622/TYY5622-17092021-0106/inspect_summary.pdf
29 TYY5622-20092021-0106


6it [00:57,  9.58s/it]


Saving figure to /adata/projects/autopi_ca1/TYY5622/TYY5622-20092021-0106/inspect_power_spectra.pdf
Saving figure to /adata/projects/autopi_ca1/TYY5622/TYY5622-20092021-0106/inspect_signal.pdf
/adata/projects/autopi_ca1/TYY5622/TYY5622-20092021-0106/inspect_summary.pdf


# Merge the results

We will merge the inspect_summary.pdf of all sessions into a big one.

In [11]:
[os.path.exists(s.path+"/inspect_summary.pdf") for s in sSesLocal]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [12]:
input_files = [s.path+"/inspect_summary.pdf" for s in sSesList]
output_file = myProject.dataPath+"/results/inspect_summary.pdf"
merge_pdf_files(input_files,output_file)

/adata/projects/autopi_ca1/results/inspect_summary.pdf
